In [1]:
import pandas as pd
import numpy as np
from dateutil.relativedelta import relativedelta
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("../preprocessed-data.csv")
df

,Date,Value
0,2010-01-01,388.91
1,2010-02-01,390.41
2,2010-03-01,391.37
3,2010-04-01,392.67
4,2010-05-01,393.21
...,...,...
179,2024-12-01,425.40
180,2025-01-01,426.65
181,2025-02-01,427.09
182,2025-03-01,428.15


In [3]:
def findOptimalSarima(timeSeries, maxP=5, maxD=5, maxQ=5, maxSeasonalP=5, maxSeasonalD=5, maxSeasonalQ=5, seasonalPeriod=12):
    bestAic = np.inf
    bestOrder = None
    bestSeasonalOrder = None
    
    print("Finding optimal SARIMA parameters...")
    
    for p in range(maxP + 1):
        for d in range(maxD + 1):
            for q in range(maxQ + 1):
                for P in range(maxSeasonalP + 1):
                    for D in range(maxSeasonalD + 1):
                        for Q in range(maxSeasonalQ + 1):
                            try:
                                model = SARIMAX(timeSeries, 
                                              order=(p, d, q), 
                                              seasonal_order=(P, D, Q, seasonalPeriod))
                                modelFit = model.fit(disp=False)
                                
                                if modelFit.aic < bestAic:
                                    bestAic = modelFit.aic
                                    bestOrder = (p, d, q)
                                    bestSeasonalOrder = (P, D, Q, seasonalPeriod)
                                    
                            except:
                                continue
    
    print(f"Best SARIMA order: {bestOrder}")
    print(f"Best seasonal order: {bestSeasonalOrder}")
    print(f"Best AIC: {bestAic}")
    return bestOrder, bestSeasonalOrder

In [4]:
def evaluateModel(df, predCol):
    # remove rows with NaN predictions for evaluation
    validDf = df.dropna(subset=[predCol, 'Value'])
    
    if len(validDf) == 0:
        return "No valid predictions to evaluate"
    
    actual = validDf['Value']
    predicted = validDf[predCol]
    
    # calculate metrics
    mae = np.mean(np.abs(actual - predicted))
    mse = np.mean((actual - predicted) ** 2)
    rmse = np.sqrt(mse)
    mape = np.mean(np.abs((actual - predicted) / actual)) * 100
    
    return f"MAE: {mae:.4f}, MSE: {mse:.4f}, RMSE: {rmse:.4f}, MAPE: {mape:.4f}%"

In [5]:
def execSarima(steps=12):
    dfCopy = df.copy()
    dfCopy['Date'] = pd.to_datetime(dfCopy['Date'])
    dfCopy.set_index('Date', inplace=True)
    
    timeSeries = dfCopy['Value']
    
    # TODO: RUN THIS LATER, WHEN CHILLIN
    # # find optimal parameters
    # bestOrder, bestSeasonalOrder = findOptimalSarima(timeSeries)
    
    # fit SARIMA model
    # model = SARIMAX(timeSeries, 
    #                order=bestOrder, 
    #                seasonal_order=bestSeasonalOrder)
    # modelFit = model.fit(disp=False)
    
    
    # temporary param, need to run top one later
    model = SARIMAX(timeSeries, 
                   order=(2,1,2), 
                   seasonal_order=(1,1,1,12))
    modelFit = model.fit(disp=False)
    

    print(modelFit.summary())
    
    inSamplePred = modelFit.fittedvalues
    
    inSamplePred.iloc[0] = np.nan
    
    resultDf = dfCopy.copy()
    resultDf['SarimaPred'] = inSamplePred
    
    forecast = modelFit.forecast(steps=steps)
    
    lastDate = dfCopy.index[-1]
    futureDates = []
    futureValues = []
    futureSarima = []
    
    for i in range(steps):
        newDate = lastDate + relativedelta(months=i+1)
        futureDates.append(newDate)
        futureValues.append(np.nan)
        futureSarima.append(forecast.iloc[i] if hasattr(forecast, 'iloc') else forecast[i])
    
    # create future dataframe
    futureDf = pd.DataFrame({
        'Value': futureValues,
        'SarimaPred': futureSarima
    }, index=futureDates)
    
    extendedDf = pd.concat([resultDf, futureDf])
    
    print("\nModel Evaluation:")
    evaluation = evaluateModel(resultDf.reset_index(), "SarimaPred")
    print(evaluation, "\n")
    
    # reset index to match MVA format
    extendedDf = extendedDf.reset_index()
    extendedDf = extendedDf.rename(columns={'index': 'Date'})
    extendedDf['Date'] = extendedDf['Date'].dt.strftime('%Y-%m-%d')
    
    print(extendedDf.to_string())
    return extendedDf

In [6]:
sarimaResults = execSarima(12)

outPath = "../sarima-results.csv"
sarimaResults.to_csv(outPath, index=False)

sarimaResults

/home/alie/programs/miniconda3/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
/home/alie/programs/miniconda3/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
/home/alie/programs/miniconda3/lib/python3.12/site-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/home/alie/programs/miniconda3/lib/python3.12/site-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'


                                      SARIMAX Results                                       
Dep. Variable:                                Value   No. Observations:                  184
Model:             SARIMAX(2, 1, 2)x(1, 1, [1], 12)   Log Likelihood                 -73.594
Date:                              Fri, 30 May 2025   AIC                            161.188
Time:                                      14:23:30   BIC                            183.180
Sample:                                  01-01-2010   HQIC                           170.111
                                       - 04-01-2025                                         
Covariance Type:                                opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1         -0.2097     50.879     -0.004      0.997     -99.930      99.510
ar.L2          0.11

,Date,Value,SarimaPred
0,2010-01-01,388.91,NaN
1,2010-02-01,390.41,388.909984
2,2010-03-01,391.37,390.409996
3,2010-04-01,392.67,391.369999
4,2010-05-01,393.21,392.670000
...,...,...,...
191,2025-12-01,NaN,427.827729
192,2026-01-01,NaN,429.029122
193,2026-02-01,NaN,429.906072
194,2026-03-01,NaN,430.643040
